# Exercises I
Let's review the steps we take at the beginning of each new module.

1. Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
2. Clone this repository within your local codeup-data-science directory.
3. Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
4. Ceate a README.md file that outlines the contents and purpose of your repository.
5. Add, commit, and push these two files.
6. Now you can add your env.py file to this repository to access the Codeup database server.
7. For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.
8. As always, add, commit, and push your work often.


# Exercises II
Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

**As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.**

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

1. Acquire `bedroomcnt`, `bathroomcnt`, `calculatedfinishedsquarefeet`, `taxvaluedollarcnt`, `yearbuilt`, `taxamount`, and `fips` from the zillow database for all 'Single Family Residential' properties.

In [1]:
from env import gdb

In [2]:
df = gdb('zillow','''
        SELECT 	bedroomcnt beds, 
		bathroomcnt baths, 
		calculatedfinishedsquarefeet sqft, 
        taxvaluedollarcnt tax_appraisal, 
        yearbuilt yr_built,
        taxamount taxes,
        fips
        FROM properties_2017;
''')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   beds           float64
 1   baths          float64
 2   sqft           float64
 3   tax_appraisal  float64
 4   yr_built       float64
 5   taxes          float64
 6   fips           float64
dtypes: float64(7)
memory usage: 159.4 MB


In [5]:
df.to_pickle('zillow_data')

In [8]:
df.size , df.shape

(20896519, (2985217, 7))

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your `wrangle.ipynb` file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [7]:
# That's a lot of beds and baths...
#   - But, none of thecounts match up to each other... NULLS!!
df.describe().T

,count,mean,std,min,25%,50%,75%,max
beds,2982272.0,3.093362,1.271569,0.00,2.00,3.00,4.0,2.500000e+01
baths,2982260.0,2.216196,1.078250,0.00,2.00,2.00,3.0,3.200000e+01
sqft,2940120.0,1831.455292,1954.198051,1.00,1215.00,1574.00,2140.0,9.525760e+05
tax_appraisal,2950951.0,443527.931066,816336.627054,1.00,188220.00,321161.00,514072.0,3.196225e+08
yr_built,2937384.0,1964.437713,23.644275,1801.00,1950.00,1963.00,1981.0,2.016000e+03
taxes,2962465.0,5408.948842,9675.566706,0.24,2468.62,4007.62,6230.5,3.823176e+06
fips,2982285.0,6048.053490,20.240825,6037.00,6037.00,6037.00,6059.0,6.111000e+03


In [9]:
# Let's check it out 
#   - .snull() followed by .sum, with (axis=0) gives us the sum of the True nulls across each column
df.isnull().sum(axis=0)

# And we have 48K Houses without a know year built, yay.

beds              2945
baths             2957
sqft             45097
tax_appraisal    34266
yr_built         47833
taxes            22752
fips              2932
dtype: int64

In [10]:
# If we run it down the rows instead...
#   - We would get a big long DataFrame 3M rows long with the number of True nulls in each row
# Instead:
#   - Let's .value_counts() - giving us the number of rows that have each value from 0-7 True nulls
df.isnull().sum(axis=1).value_counts(dropna=False)

# And we see 2.9M good full rows... looks like some nice low hanging fruit to me
#  in total...

0    2901918
2      36500
1      33723
3       9045
7       2932
4       1095
5          4
dtype: int64

In [17]:
# We want to know how many cells we're dropping if we do it quick and dirty...

# Save it to a variable and .sum() from the 1st index down to catch everything without 0 == True nulls
num_nulls = df.isnull().sum(axis=1).value_counts()
num_nulls[1:].sum()

83299

In [11]:
# Let's look into saving some of these and what that would look like
#   - There are 34K with only 1 null, any chance they're all the same? 🤞🏻

df[df.isnull().sum(axis=1) == 1]

# Doesn't look like it...

,beds,baths,sqft,tax_appraisal,yr_built,taxes,fips
22,0.0,0.0,1442.0,50689.0,NaN,1522.08,6037.0
34,3.0,3.0,2019.0,1431502.0,NaN,17034.34,6037.0
104,0.0,0.0,3725.0,587254.0,NaN,8253.47,6037.0
121,0.0,0.0,1120.0,NaN,1981.0,285.84,6037.0
122,0.0,0.0,1488.0,NaN,1978.0,399.58,6037.0
...,...,...,...,...,...,...,...
2981778,1.0,1.0,600.0,214264.0,1987.0,NaN,6037.0
2981849,5.0,3.0,2830.0,386686.0,1906.0,NaN,6037.0
2981939,0.0,0.0,NaN,74300.0,2014.0,793.96,6111.0
2981970,0.0,0.0,NaN,29100.0,2005.0,250.46,6111.0


In [29]:
# Check where df.isnull() == True, .sum(axis=1) == 1 to sum across the columns and check for just 1 True null
#   Then to look down the columns where there's only 1 null per row, and count the nulls across the columns

df[df.isnull().sum(axis=1) == 1].isnull().sum(axis=0)

beds                 0
baths                0
sqft              1039
tax_appraisal    24016
yr_built          4443
taxes             4225
fips                 0
dtype: int64

**We see that 24K of the nulls are tax_apraisal... might be important for finding value**

 Maybe year built is something we can just guess at?

- from the surrounding area homes with the same fips? Save 4K there 
<br>
- Then guess taxes from the mean tax from tax apraisal amount... Save 4K there
<br>
- What's the chance we can get the tax_apraisal amount from taxes... Save 24K more

 That's for another day ;)

In [22]:
# I'm droping 83K rows from 3M... which is:
num_nulls[1:].sum() / num_nulls.sum()

0.02790383412663133

In [23]:
# So if that's 2.8% and I'm talking about saving 32K...
32_000 / num_nulls.sum()

#  Which save another 1% and gets us down to dropping just 1.8% of the data

#  Maybe worth it, but it's for another go

0.010719488733984832

In [24]:
# Is that the shape we're looking for?
df.dropna().shape

# looks good to me...

(2901918, 7)

In [26]:
# We'll save the full dataframe to df_full to play around with at a later time
df_full = df

# set it equal to itself with all null rows dropped!
df = df.dropna()

# And see what it looks like
df.describe().T

,count,mean,std,min,25%,50%,75%,max
beds,2901918.0,3.170278,1.185457,0.00,2.00,3.00,4.00,2.500000e+01
baths,2901918.0,2.271617,1.030979,0.00,2.00,2.00,3.00,3.200000e+01
sqft,2901918.0,1836.479432,1935.554085,1.00,1218.00,1581.00,2148.00,9.525760e+05
tax_appraisal,2901918.0,444828.167789,730804.472995,22.00,192600.00,324450.00,516903.00,2.870985e+08
yr_built,2901918.0,1964.258716,23.626704,1801.00,1950.00,1963.00,1981.00,2.016000e+03
taxes,2901918.0,5456.144079,8740.155378,5.04,2543.66,4059.52,6277.86,3.458861e+06
fips,2901918.0,6047.878252,20.090158,6037.00,6037.00,6037.00,6059.00,6.111000e+03


In [9]:
# There's still zeros in theh beds and baths columns, that's not right.
no_beds_baths_index = df[(df.beds == 0) | (df.baths == 0)].index
len(no_beds_baths_index)

# Well... darn. Guess I'm dropping another 1.5% ish...

46615

In [8]:
# I like these statistics a lot more... and feel good about the data integrity
# I'd like to look into regaining some of my data using a couple of methods...
df.drop(index= no_beds_baths_index).describe().T

,count,mean,std,min,25%,50%,75%,max
beds,2855303.0,3.220891,1.124533,1.00,3.00,3.00,4.00,25.00
baths,2855303.0,2.305752,1.000843,0.50,2.00,2.00,3.00,32.00
sqft,2855303.0,1805.249165,1094.429550,1.00,1218.00,1577.00,2134.00,952576.00
tax_appraisal,2855303.0,441274.149034,607584.970057,22.00,193504.00,324572.00,516000.00,90188462.00
yr_built,2855303.0,1964.371389,23.566793,1801.00,1950.00,1963.00,1981.00,2016.00
taxes,2855303.0,5408.772482,7025.109525,12.57,2552.16,4059.66,6264.23,1078101.87
fips,2855303.0,6047.729691,19.909250,6037.00,6037.00,6037.00,6059.00,6111.00


# Regaining data:<br>
**1. Use the squarefootage to model the number of bedrooms and bathrooms based on logistic regression model**
<br><br>
**2. Use tax appraisal data to approximate the taxes**
<br><br>
**3. Use taxes to approximate the tax appraisal (2 and 3 are the opposite cases when the opposite info is missing)**
<br><br>
**4. Check for patterns in the year built from index (this is a case where that could be true) or from fips...**
  - Alternately, we might go back and grab the zip codes to better approximate year built
  - Alternately still, we might find that year built doesn't affect value much...
<br>

**5. These are notes for the ETL lover that lives somewhere and gets some satisfaction out of cleaning data**
  - Hello...?

# 3. Store all of the necessary functions:
  - to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your `wrangle.py` file. Name your final function `wrangle_zillow`.

In [1]:
# We're gonna restart the kernel here, and just run this cell...
import pandas as pd
from wrangle import wrangle_zillow

df = wrangle_zillow()

df.describe().T

,count,mean,std,min,25%,50%,75%,max
beds,2855303.0,3.220891,1.124533,1.00,3.00,3.00,4.00,25.00
baths,2855303.0,2.305752,1.000843,0.50,2.00,2.00,3.00,32.00
sqft,2855303.0,1805.249165,1094.429550,1.00,1218.00,1577.00,2134.00,952576.00
tax_appraisal,2855303.0,441274.149034,607584.970057,22.00,193504.00,324572.00,516000.00,90188462.00
yr_built,2855303.0,1964.371389,23.566793,1801.00,1950.00,1963.00,1981.00,2016.00
taxes,2855303.0,5408.772482,7025.109525,12.57,2552.16,4059.66,6264.23,1078101.87
fips,2855303.0,6047.729691,19.909250,6037.00,6037.00,6037.00,6059.00,6111.00


In [2]:
#  And it works. I still have 2.8 Million entries in my dataset.
df_full = pd.read_pickle('zillow_data')

In [5]:
df.shape[0] / df_full.shape[0]

# Which is 96% of my original dataset

0.9564808856441592

# Final cell
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>